# Multiscale MILP

This is a continuation of 'One Location, One Temporal Scale, One Operation, Linear Programming Example' [Example 1]. Refer Example 1 to learn the basics on how Energia models processes. 

In this example, we add another Process [Solar PV] and Storage [Li-ion Battery]. Technology choice is modeled using binaries. Moreover, the model is multiscale as the operational capacities are decision variables. 

In [1]:
from energia import *

m = Model('example2')
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

## Resources

### Conveniently declaring components

Use m.declare(\<Object Type\>, \<list of names\>) to declare a large number of objects in one step.

In [2]:
m.declare(Resource, ['power', 'wind', 'solar'])

### Set bounds on Resource flows

Unlike wind which has bound on the total consumption, we set a daily limit on solar energy. The same bound is repeated in each quarter. The following constraints are written.

$\mathbf{cons}_{solar, network, quarter_0} \leq 100$

$\mathbf{cons}_{solar, network, quarter_1} \leq 100$

$\mathbf{cons}_{solar, network, quarter_2} \leq 100$

$\mathbf{cons}_{solar, network, quarter_3} \leq 100$

In [3]:
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

2025-10-23 12:40:54,422 [INFO] Balance for solar in (l0, q): initializing
2025-10-23 12:40:54,423 [INFO] ✔ Completed in 0.00023627281188964844 seconds
2025-10-23 12:40:54,424 [INFO] Binding consume in domain (solar, l0, q)
2025-10-23 12:40:54,425 [INFO] ✔ Completed in 0.00016450881958007812 seconds
2025-10-23 12:40:54,425 [INFO] Balance for wind in (l0, y): initializing
2025-10-23 12:40:54,426 [INFO] ✔ Completed in 0.00011801719665527344 seconds
2025-10-23 12:40:54,426 [INFO] Binding consume in domain (wind, l0, y)
2025-10-23 12:40:54,427 [INFO] ✔ Completed in 0.0001010894775390625 seconds
2025-10-23 12:40:54,428 [INFO] Balance for power in (l0, q): initializing
2025-10-23 12:40:54,428 [INFO] ✔ Completed in 0.0001533031463623047 seconds
2025-10-23 12:40:54,428 [INFO] Binding release in domain (power, l0, q)
2025-10-23 12:40:54,429 [INFO] ✔ Completed in 0.00018668174743652344 seconds


## Operations 

### Capacity as a variable 

Here we want the optimization problem to determine the optimal capacity. Moreover, we set binaries to avoid the lower bound being adhered to if the process is not set up. 

If the bounds are meant to be compulsory limits, skip the .x 

In [4]:
m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.capacity.show()

2025-10-23 12:40:54,435 [INFO] Binding capacity in domain (wf, l0, y)
2025-10-23 12:40:54,435 [INFO] ✔ Completed in 0.00021958351135253906 seconds
2025-10-23 12:40:54,436 [INFO] Binding capacity in domain (wf, l0, y)
2025-10-23 12:40:54,437 [INFO] ✔ Completed in 0.00023174285888671875 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Unlike in Example 1, where the capacity was know, capacity is a variable here. 

Moreover, the expenditure associated with operating and capacitating are different

In [5]:
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49
m.operate.show(True)

2025-10-23 12:40:54,447 [INFO] Binding operate in domain (wf, l0, q)
2025-10-23 12:40:54,447 [INFO] ✔ Completed in 0.0002779960632324219 seconds
2025-10-23 12:40:54,449 [INFO] Mapping operate: (wf, l0, q) → (wf, l0, y)
2025-10-23 12:40:54,450 [INFO] ✔ Completed in 0.0001456737518310547 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10

m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

2025-10-23 12:40:54,461 [INFO] Binding capacity in domain (pv, l0, y)
2025-10-23 12:40:54,461 [INFO] ✔ Completed in 0.0002353191375732422 seconds
2025-10-23 12:40:54,462 [INFO] Binding capacity in domain (pv, l0, y)
2025-10-23 12:40:54,463 [INFO] ✔ Completed in 0.00020384788513183594 seconds
2025-10-23 12:40:54,463 [INFO] Binding operate in domain (pv, l0, q)
2025-10-23 12:40:54,464 [INFO] ✔ Completed in 0.00026917457580566406 seconds
2025-10-23 12:40:54,466 [INFO] Mapping operate: (pv, l0, q) → (pv, l0, y)
2025-10-23 12:40:54,467 [INFO] ✔ Completed in 0.0001461505889892578 seconds


### Storage Operation

energia now allows storing to require the use of other resources, example power for hydrogen cryogenic storage. 

Provide an equation similar to Process, in this case the basis is the stored resource 
If no other resource is provided, it is assumed to be the charging/discharging efficiency

Note that the following are created internally: 
1. auxilary resource  with name resource.stored 
2. charging and discharging processes as storage.charge and storage.discharge 

The parameters for each of these can be set individually, thus allowing for a wide range of modeling approaches 

In [7]:
m.lii = Storage()

m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10

# m.lii.capacity >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432

m.lii.inventory[m.usd.spend] == 2000


2025-10-23 12:40:54,473 [INFO] Binding invcapacity in domain (lii.stored, l0, y)
2025-10-23 12:40:54,474 [INFO] ✔ Completed in 0.0003399848937988281 seconds
2025-10-23 12:40:54,474 [INFO] Binding invcapacity in domain (lii.stored, l0, y)
2025-10-23 12:40:54,475 [INFO] ✔ Completed in 0.0002455711364746094 seconds
2025-10-23 12:40:54,476 [INFO] Balance for lii.stored in (l0, y): initializing


## Locating Operations

Operations can be located as 

operation.locate(\<list of locations\>)

or 

m.location.operations(\<list of operations\>)

They both do the same thing 

In [8]:
m.pem = Process()

m.pem(-m.power) == m.solar 

η(pem, power)

In [9]:
m.pv.locate(m.network)
m.network.locate(m.wf, m.lii)

2025-10-23 12:40:54,489 [INFO] Balance for power in (l0, q): adding produce(power, l0, q, operate, pv)
2025-10-23 12:40:54,490 [INFO] ✔ Completed in 0.000202178955078125 seconds
2025-10-23 12:40:54,491 [INFO] Balance for solar in (l0, q): adding expend(solar, l0, q, operate, pv)
2025-10-23 12:40:54,492 [INFO] ✔ Completed in 0.0003762245178222656 seconds
2025-10-23 12:40:54,493 [INFO] Balance for power in (l0, q): adding produce(power, l0, q, operate, wf)
2025-10-23 12:40:54,494 [INFO] ✔ Completed in 0.00018644332885742188 seconds
2025-10-23 12:40:54,495 [INFO] Balance for wind in (l0, y): adding expend(wind, l0, y, operate, wf)
2025-10-23 12:40:54,495 [INFO] ✔ Completed in 0.00014591217041015625 seconds
2025-10-23 12:40:54,496 [INFO] Assuming  lii.stored inventory capacity is unbounded in (l0, y)
2025-10-23 12:40:54,496 [INFO] Assuming inventory of lii.stored is unbounded in (l0, y)
2025-10-23 12:40:54,497 [INFO] Assuming inventory of lii.stored is bound by inventory capacity in (l0, q

## Inventory Balance

Inventory is passed on from one time period (t - 1) to the next (t) and hence features in the general resource balance for resource.stored 

In [10]:
m.inventory.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Optimize!

In [11]:
m.usd.spend.opt()

2025-10-23 12:40:54,525 [INFO] Mapping spend: (usd, l0, y, capacity, wf) → (usd, l0, y)
2025-10-23 12:40:54,526 [INFO] ✔ Completed in 0.0001647472381591797 seconds
2025-10-23 12:40:54,527 [INFO] Mapping spend: (usd, l0, y, operate, wf) → (usd, l0, y)
2025-10-23 12:40:54,527 [INFO] ✔ Completed in 0.0001392364501953125 seconds
2025-10-23 12:40:54,527 [INFO] Mapping spend: (usd, l0, y, capacity, pv) → (usd, l0, y)
2025-10-23 12:40:54,528 [INFO] ✔ Completed in 0.00015878677368164062 seconds
2025-10-23 12:40:54,529 [INFO] Mapping spend: (usd, l0, y, operate, pv) → (usd, l0, y)
2025-10-23 12:40:54,529 [INFO] ✔ Completed in 0.00013709068298339844 seconds
2025-10-23 12:40:54,530 [INFO] Mapping spend: (usd, l0, y, invcapacity, lii.stored) → (usd, l0, y)
2025-10-23 12:40:54,530 [INFO] ✔ Completed in 0.00011944770812988281 seconds
2025-10-23 12:40:54,531 [INFO] Mapping spend: (usd, l0, y, inventory, lii.stored) → (usd, l0, y)
2025-10-23 12:40:54,531 [INFO] ✔ Completed in 0.00013256072998046875 se

Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-01
Read MPS format model from file Program(example2).mps
Reading time = 0.01 seconds
PROGRAM(EXAMPLE2): 83 rows, 78 columns, 196 nonzeros


2025-10-23 12:40:54,544 [INFO] Optimizing Program(example2) using gurobi


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 83 rows, 78 columns and 196 nonzeros
Model fingerprint: 0xc4ff6a31
Variable types: 75 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 4e+02]
Presolve removed 71 rows and 66 columns
Presolve time: 0.00s
Presolved: 12 rows, 12 columns, 34 nonzeros
Variable types: 12 continuous, 0 integer (0 binary)

Root relaxation: objective 3.006497e+08, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3.006497e+08 3.0065e+08  0.0

2025-10-23 12:40:54,557 [INFO] Solution found. Use .output() to display it
2025-10-23 12:40:54,557 [INFO] Creating Solution object, check.solution


## Solution

### Inventory Profiles

The inventory maintained in each time period is:

In [12]:
m.inventory.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount charged into inventory is:

In [13]:
m.produce(m.power.lii, m.lii.charge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount discharged from inventory is:

In [14]:
m.produce(m.power, m.lii.discharge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Integer Decisions 

All the operations are setup in this case

In [15]:
m.capacity.reporting.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [16]:
m.capacity.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [17]:
m.show(True)

# Mathematical Program for Program(example2)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>